In [20]:
import pandas as pd
import ast
import numpy as np

In [31]:
path = 'consensus - full.csv'
df = pd.read_csv(path)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [32]:
df['hypothesis'] = df['hypothesis'].apply(ast.literal_eval)

In [33]:
df.rationale = df.rationale.astype(str)
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [34]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,"""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...","Questions: ""There are 10 apples on an apple tr...",1. The commonly agreed-upon answer to the ques...,1. the commonly agreed-upon answer to the ques...


In [35]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [36]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [37]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer


In [38]:
# read paper's answer
paper = pd.read_csv('paper.csv', index_col=0)

In [39]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)
paper.index = paper.index.str.strip()

In [40]:
df.set_index('premise', inplace=True)

In [41]:
paper['my_label'] = df['LLM_answer']

In [42]:
sum(paper.label == paper.my_label) / len(paper)

0.7395544605276666

In [43]:
df.to_csv('consensus - full.csv')

In [22]:
TYPE = 'consensus'
temp = pd.read_csv(f'{TYPE} - full.csv')

import re
temp[~temp['rationale'].apply(lambda x: bool(re.match(r'\d+\.', x.strip())))]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer
